In [7]:
import json
import copy
from datetime import datetime
import configparser
import os
import yaml
import random

from openai import OpenAI

print("Current working directory:", os.getcwd())

config = configparser.ConfigParser()
# Adjust path to look for config.ini in AImodule regardless of the current directory
base_dir = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
config_path = os.path.join(base_dir, 'config.ini')
print(config_path)
config.read(config_path)

print("Config sections found:", config.sections())

if 'OpenAI' not in config:
    print("Error: 'OpenAI' section not found in config.ini")
openai_key = config['OpenAI']['chatgpt_key']
# deepseek_key1 = config['OpenAI']['deepseek_key']
# deepseek_key2 = config['OpenAI']['deepseek_key2']
# deepseek_key = random.choice([
#     config['OpenAI']['deepseek_key'],
#     config['OpenAI']['deepseek_key2']
# ])
deepseek_key=config['OpenAI']['deepseek_key']

is_chatgpt = config['OpenAI'].getboolean('useChatGPT', fallback=True)
if is_chatgpt:
    client = OpenAI(api_key=openai_key)
    client_embedding = OpenAI(api_key=openai_key)
    model_small = "gpt-4o-mini"
    model_large = "gpt-4o"
else:
    # client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=deepseek_key) 
    client = OpenAI(base_url="https://api.deepseek.com", api_key=deepseek_key) 
    client_embedding = OpenAI(api_key=openai_key)
    # model_small = "deepseek/deepseek-r1-distill-llama-70b"
    # model_large = "deepseek/deepseek-r1-distill-llama-70b"
    model_small = "deepseek-chat"
    model_large = "deepseek-chat"


yaml_path = os.path.join(base_dir, 'char_config.yaml')

# Load the YAML file
with open(yaml_path, 'r', encoding='utf-8' ) as file:
    char_config = yaml.safe_load(file)
    print("YAML content loaded successfully.")


def get_npc_descriptions(from_npcId=None):
    """
    Returns a string with each NPC's name and a brief part of their description,
    one per line, in the format: "- Name, Description (first sentence)."
    """
    # Define NPC groups
    group1 = {10016, 10017, 10018, 10019, 10020, 10021}
    group2 = {10012, 10009, 10006}

    # Determine allowed NPC IDs based on from_npcId
    if from_npcId is not None:
        if from_npcId in group1:
            allowed_ids = group1
        elif from_npcId in group2:
            allowed_ids = group2
        else:
            allowed_ids = set()
    else:
        allowed_ids = None

    descriptions = []
    for npc in char_config.get("npcCharacters", []):
        # Skip NPCs not in the allowed set when filtering
        if allowed_ids is not None and npc.get("npcId") not in allowed_ids:
            continue
        # Get the first sentence (or a summary) from the description.
        brief_desc = npc.get("description", "").split(".")[0]
        descriptions.append(f"- {npc['name']}, {brief_desc}.")
    return "\n".join(descriptions)

def get_npc_id_mapping():
    """
    Returns a string mapping npcId to name, one per line, in the format: "npcId : Name".
    """
    mappings = []
    for npc in char_config.get("npcCharacters", []):
        mappings.append(f"{npc['npcId']} : {npc['name']}")
    return "\n".join(mappings)

Current working directory: /Users/jackhan/Desktop/MultiAgent-Framwork/MultiAgent-Framwork/BhrCtrl
/Users/jackhan/Desktop/MultiAgent-Framwork/MultiAgent-Framwork/config.ini
Config sections found: ['mysql', 'Paths', 'OpenAI', 'NetworkSocket']
YAML content loaded successfully.


In [10]:
get_npc_descriptions(10017)

'- Leo, Leo, the introspective artist of DraMai Town, crafts silence into strokes of color.\n- Nova, Nova, the spirited farmer of DraMai Town, infuses vitality into every inch of her land.\n- Aiden, Aiden, DraMai Town’s golden athlete, carries the glow of past triumphs and the determination to redefine himself.\n- Ivy, Ivy, the gentle baker of DraMai Town, fills the community with the aroma of freshly baked bread and heartfelt warmth.\n- Qiu, Qiu, DraMai Town’s enigmatic investor, moves with the grace of someone used to both boardrooms and spotlight.\n- Kai, Kai, the methodical lawyer of DraMai Town, upholds justice with both precision and compassion.'

In [6]:
get_npc_id_mapping()

'10012 : Trump\n10009 : Elon Musk\n10006 : Satoshi\n10016 : Leo\n10020 : Nova\n10021 : Aiden\n10017 : Ivy\n10019 : Qiu\n10018 : Kai'